# Train ReLayNet

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.autograd import Variable
import h5py

from relaynet_pytorch.relay_net import ReLayNet
from relaynet_pytorch.data_utils import get_imdb_data
from helper import * # import helper functions

#torch.set_default_tensor_type('torch.FloatTensor')

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [2]:
train_data, test_data = get_imdb_data()
print("Train size: %i" % len(train_data))
print("Test size: %i" % len(test_data))

Train size: 480
Test size: 120


In [3]:
# Printing Errors for CUDA
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
CUDA_LAUNCH_BLOCKING=1

In [4]:
from relaynet_pytorch.relay_net import ReLayNet
from relaynet_pytorch.solver import Solver

bs = 20
wrkrs = 4
train_loader = torch.utils.data.DataLoader(train_data, batch_size=bs, shuffle=True, num_workers=wrkrs)
val_loader = torch.utils.data.DataLoader(test_data, batch_size=bs, shuffle=False, num_workers=wrkrs)

param ={
        'num_channels':1,
        'num_filters':64,
        'kernel_h':3,
        'kernel_w':7,
        'kernel_c': 1,
        'stride_conv':1,
        'pool':2,
        'stride_pool':2,
        'num_class':8
    }

exp_dir_name = 'RawOCT4'

relaynet_model = ReLayNet(param)
solver = Solver(optim_args={"lr": 1e-2})
solver.train(relaynet_model, train_loader, val_loader, log_nth=1, num_epochs=2, exp_dir_name=exp_dir_name)

START TRAIN.
[Iteration : 0/2] - Train Loss: 34.09086608886719
Per class average dice score is 0.349950199821
[Iteration : 0/2] - Train Loss: 29.25154685974121
Per class average dice score is 0.323851216999
[Iteration : 0/2] - Train Loss: 25.33164405822754
Per class average dice score is 0.324186536405
[Iteration : 0/2] - Train Loss: 23.67253875732422
Per class average dice score is 0.319467290727
[Iteration : 0/2] - Train Loss: 21.61916732788086
Per class average dice score is 0.322135896581
[Iteration : 0/2] - Train Loss: 19.685718536376953
Per class average dice score is 0.361377563575
[Iteration : 0/2] - Train Loss: 19.057113647460938
Per class average dice score is 0.343369008685
[Iteration : 0/2] - Train Loss: 19.53956413269043
Per class average dice score is 0.343961320655
[Iteration : 0/2] - Train Loss: 18.379056930541992
Per class average dice score is 0.338568563596
[Iteration : 0/2] - Train Loss: 17.337627410888672
Per class average dice score is 0.588602014269
[Iteration : 

In [ ]:
SEG_LABELS_LIST = [
#     {"id": -1, "name": "void", "rgb_values": [0, 0, 0]},
    {"id": 0, "name": "void", "rgb_values": [0, 0, 0]}, # black
    {"id": 1, "name": "Myocardium", "rgb_values": [255,0,0]}, # red
    {"id": 2, "name": "Endocardium", "rgb_values": [0, 0, 255]}, # blue
    {"id": 3, "name": "Fibrosis", "rgb_values": [177,10,255]}, # purple
    {"id": 4, "name": "Fat", "rgb_values": [0, 255, 0]}, # green
    {"id": 5, "name": "Dense Collagen", "rgb_values": [255, 140, 0]}, # orange
    {"id": 6, "name": "Loose Collagen", "rgb_values": [255, 255, 0]}, # yellow
    {"id": 7, "name": "Smooth Muscle", "rgb_values": [255,0,255]}# magenta/pink
]; 

def label_img_to_rgb(label_img):
    label_img = np.squeeze(label_img)
    labels = np.unique(label_img)
    label_infos = [l for l in SEG_LABELS_LIST if l['id'] in labels]

    label_img_rgb = np.array([label_img,
                              label_img,
                              label_img]).transpose(1,2,0)
    for l in label_infos:
        mask = label_img == l['id']
        label_img_rgb[mask] = l['rgb_values']

    return label_img_rgb.astype(np.uint8)

In [9]:
import matplotlib.pyplot as plt
import torch.nn.functional as F

relaynet_model =  torch.load('models/RawOCT2/relaynet_epoch99.model')

out = relaynet_model(Variable(torch.Tensor(test_data.X[10:11]).cuda(),volatile=True))
out = F.softmax(out,dim=1)
max_val, idx = torch.max(out,1)
idx = idx.data.cpu().numpy()
idx = label_img_to_rgb(idx)
plt.imshow(idx)
plt.show()

img_test = test_data.X[10:11]
img_test = np.squeeze(img_test)
plt.imshow(img_test)
plt.show()

AttributeError: 'ReLayNet' object has no attribute 'train_loss_history'